In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers==2.2.2
!pip install simpletransformers==0.13.0
!pip install seqeval==0.0.12
!pip install torch==1.4+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html --no-cache-dir
!rm -rf .git
!rm -rf apex
!git clone https://github.com/NVIDIA/apex
!git checkout 494f8ab3fc1b0b26949a3bcbb2bcac78008d48c1
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Cloning into 'apex'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (312/312), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 6147 (delta 227), reused 204 (delta 140), pack-reused 5835
Receiving objects: 100% (6147/6147), 13.64 MiB | 7.03 MiB/s, done.
Resolving deltas: 100% (4019/4019), done.
Checking out files: 100% (249/249), done.
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-txxnv2qy
Created temporary directory: /tmp/pip-req-tracker-bfugxzk7
Created requirements tracker '/tm

In [2]:
cd /content/drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [4]:
import numpy as np
import pandas as pd
import torch
import gc
import csv

from CsvReader import ReadCsv
from HelperFunctions import ShuffleData, replace_all, ReplaceSentimentsWithIndexes, SplitArticlesIntoSentiments, getMetrics, TransformDataIntoDataframe, CalculateWeights
from TrainEvalModel import TrainModelForMultiClass, EvalFromModel

data, categories, tendencies = ReadCsv('/content/drive/My Drive/Colab Notebooks Pascal/bert-etiki/etiki-data','test.csv','companies.csv', 'categories.csv','references.csv','tendencies.csv', 'topics.csv')
sentimentData = ReplaceSentimentsWithIndexes(data[:,[0,7,13]])
labels = np.unique(sentimentData[:,1])
weights = CalculateWeights(labels,sentimentData)

for i in range(1,5):
  for j in range(5,6):
    positiveArticles, negativeArticles, controversialArticles = SplitArticlesIntoSentiments(sentimentData)
    positiveArticles = ShuffleData(positiveArticles)
    negativeArticles = ShuffleData(negativeArticles)
    controversialArticles = ShuffleData(controversialArticles)

    trainData = np.vstack((positiveArticles[:int(len(positiveArticles)*0.7)],controversialArticles[:int(len(controversialArticles)*0.7)],negativeArticles[:int(len(negativeArticles)*0.7)]))
    testData = np.vstack((positiveArticles[int(len(positiveArticles)*0.7):],controversialArticles[int(len(controversialArticles)*0.7):],negativeArticles[int(len(negativeArticles)*0.7):]))
    train_df = TransformDataIntoDataframe(trainData)
    eval_df = TransformDataIntoDataframe(testData)
#------------------------------XLNet------------------------------
    algo = 'xlnet'        
    args = {'reprocess_input_data': True,
              'overwrite_output_dir': True,
              'num_train_epochs':j ,
              'use_cached_eval_features':True,
              'silent':True
              }
#--------------------------non-weighted---------------------------
    model = TrainModelForMultiClass(algo, 'xlnet-base-cased',train_df,3,args)
    result, model_outputs, wrong_predictions = EvalFromModel(model,eval_df)

    with open('results/weight/'+algo+'-noweight-mcc.csv', 'a', newline='') as f:
      writer = csv.writer(f)
      writer.writerow([result['mcc'],result["acc"],result["eval_loss"]])

    getMetrics(result["matr"],3,'weight/'+algo+'-noweight') 

#---------------------------weighted-----------------------------
    model = TrainModelForMultiClass(algo, 'xlnet-base-cased',train_df,3,args,weights)
    result, model_outputs, wrong_predictions = EvalFromModel(model,eval_df)

    with open('results/weight/'+algo+'-weight-mcc.csv', 'a', newline='') as f:
      writer = csv.writer(f)
      writer.writerow([result['mcc'],result["acc"],result["eval_loss"]])

    getMetrics(result["matr"],3,'weight/'+algo+'-weight') 

Converting to features started.
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:113: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Training of xlnet model complete. Saved to outputs/.
Converting to features started.
Confusion Matrix: [[71  0  3]
 [ 7  2  7]
 [ 6  2 50]]
Precision: [0.8452381  0.5        0.83333333]
Recall: [0.95945946 0.125      0.86206897]
F1-Score: [0.89873418 0.2        0.84745763]
Avg Precision: 0.7261904761904762
Avg Recall: 0.648842808325567
Avg F1-Score: 0.6487306014446113
Converting to features started.
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
kee

/content/drive/My Drive/Colab Notebooks/HelperFunctions.py:173: RuntimeWarning: invalid value encountered in true_divide
  foneScore =  (2 *(precision*recall)/(precision+recall))


Confusion Matrix: [[66  2  6]
 [10  0  6]
 [ 7  1 50]]
Precision: [0.79518072 0.         0.80645161]
Recall: [0.89189189 0.         0.86206897]
F1-Score: [0.84076433        nan 0.83333333]
Avg Precision: 0.5338774452649306
Avg Recall: 0.5846536191363777
Avg F1-Score: nan
Converting to features started.
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Gradient 